<h1><font color="#113D68" size=6>PermGrad: Interpretable Hybrid Neural Networks with Synthetic Images for Tabular Data</font></h1>

---

# <font color="#004D7F" size=6> 1. Libraries</font>

---
# <font color="#004D7F" size=5> 1.1. System setup</font>

```
    sudo pip3 install tensorflow==2.17.1 torchmetrics pytorch_lightning TINTOlib==0.0.26 imblearn keras_preprocessing mpi4py bitstring optuna
```

---
# <font color="#004D7F" size=5> 1.2. Invoke the libraries</font>

In [ ]:
import gc
import math
import os

import cv2
import keras
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.patches as mpatches
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
from tqdm import tqdm

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

os.environ["KERAS_BACKEND"] = "tensorflow"

import tensorflow as tf

from keras.layers import (
    Dense,
    Conv2D,
    Flatten,
    Dropout,
    MaxPooling2D,
    BatchNormalization,
    Input,
    Concatenate,
)
from keras.utils import plot_model

from tensorflow.keras.initializers import (
    HeNormal,
)
from tensorflow.keras.layers import (
    Lambda
)
from tensorflow.keras.models import (
    Model,
    load_model
)
from TINTOlib.tinto import TINTO

In [ ]:
np.random.seed(42)
tf.random.set_seed(42)

---
# <font color="#004D7F" size=6> 2. Data processing</font>

---
# <font color="#004D7F" size=5> 2.1. TINTOlib methods</font>

In [ ]:
dataset = "Puma"
pixels=20
problem_type = "regression"

images_folder = f"../images/{dataset}"
image_model = TINTO(problem= problem_type,blur=False, pixels=pixels)

---
# <font color="#004D7F" size=5> 2.2. Read the dataset</font>

In [ ]:
if dataset == "Puma":
  dataset_path = "../datasets/PUMA/puma8NH.csv"
  df=pd.read_csv(dataset_path, delimiter=',')
  df
  df_x = df.drop('values', axis = 1)
  df_y = df['values']

---
# <font color="#004D7F" size=5> 2.3. Generate images</font>

In [ ]:
force_recreate_images = True

if not os.path.exists(images_folder) or force_recreate_images:
    image_model.generateImages(df, images_folder)
else:
    print("The images are already generated")

img_paths = os.path.join(images_folder,problem_type+".csv")

---
# <font color="#004D7F" size=5> 2.4. Read images</font>

In [ ]:
imgs = pd.read_csv(img_paths)
imgs["images"]= images_folder + "/" + imgs["images"]

---
# <font color="#004D7F" size=5> 2.5. Preprocess the dataset</font>

In [ ]:
columns_to_normalize = df.drop(columns='values').columns

df_normalized = (df[columns_to_normalize] - df[columns_to_normalize].min()) / (df[columns_to_normalize].max() - df[columns_to_normalize].min())

df = pd.concat([df_normalized, df['values']], axis=1)

df_normalized.head(2)

In [ ]:
combined_dataset = pd.concat([imgs,df.iloc[:, :-1]],axis=1)

df_x = combined_dataset
df_y = df["values"]

print(df_y)

---
# <font color="#004D7F" size=6> 3. Pre-modelling phase</font>

---
# <font color="#004D7F" size=5> 3.1. Data curation</font>

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(df_x, df_y, test_size = 0.40, random_state = 42)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size = 0.50, random_state = 42)

X_train_num = X_train.drop("images",axis=1)
X_val_num = X_val.drop("images",axis=1)
X_test_num = X_test.drop("images",axis=1)

X_train_img = np.array([cv2.resize(cv2.imread(img),(pixels,pixels)) for img in X_train["images"]])
X_val_img = np.array([cv2.resize(cv2.imread(img),(pixels,pixels)) for img in X_val["images"]])
X_test_img = np.array([cv2.resize(cv2.imread(img),(pixels,pixels)) for img in X_test["images"]])

print("Image shape",X_train_img[0].shape)
size=X_train_img[0].shape[0]
print("Image size (pixels):", pixels)

In [ ]:
df_train = pd.concat([X_train, y_train], axis = 1)
df_test = pd.concat([X_test, y_test], axis = 1)
df_val = pd.concat([X_val, y_val], axis = 1)

In [ ]:
print(df_train.shape)
print(df_val.shape)
print(df_test.shape)

---
# <font color="#004D7F" size=6> 4. Modelling with HyNN</font>

---
# <font color="#004D7F" size=5> 4.1. HyNN</font>

In [ ]:
base_checkpoint_dir = f'../models/{dataset}/optuna_checkpoints'

model_path = f'../models/{dataset}/model_{dataset}_hybrid.keras'

study_db_path = f'../models/{dataset}/study_{dataset}_hybrid.db'

storage_url = f"sqlite:///{study_db_path}"
study_name = f"{dataset}_hybrid_study"

In [ ]:
def create_multimodal_regressor(trial, shape, pixels):
    dropout = trial.suggest_float("dropout", 0.1, 0.4)
    lr = trial.suggest_float("lr", 1e-5, 1e-2, log=True)
    n_dense_layers = trial.suggest_int("n_dense_layers", 1, 4)
    n_conv_layers = trial.suggest_int("n_conv_layers", 1, 4)
    base_filters = trial.suggest_int("base_filters", 8, 256)
    optimizer_name = trial.suggest_categorical("optimizer", ["adam", "adamw"])
    activation_fn = trial.suggest_categorical("activation", ["relu"])

    init_name = trial.suggest_categorical("initializer", ["he_normal"])
    if init_name == "he_normal":
        initializer = HeNormal()

    ff_inputs = Input(shape=(shape,))
    x_tab = ff_inputs
    for i in range(n_dense_layers):
        units = trial.suggest_int(f"dense_units_{i}", 8, 256)
        x_tab = Dense(units, activation=activation_fn, kernel_initializer=initializer)(x_tab)
        x_tab = BatchNormalization()(x_tab)
        x_tab = Dropout(dropout)(x_tab)
    ff_model = Model(ff_inputs, x_tab)

    cnn_inputs = Input(shape=(pixels, pixels, 3))
    x_cnn = cnn_inputs

    for i in range(n_conv_layers):
        filters = int(base_filters * (2 ** i))
        x_cnn = Conv2D(filters, (3, 3), activation=activation_fn, padding='same', kernel_initializer=initializer)(x_cnn)
        x_cnn = BatchNormalization()(x_cnn)
        x_cnn = MaxPooling2D(2, 2)(x_cnn)
        x_cnn = Dropout(dropout)(x_cnn)
    x_cnn = Flatten()(x_cnn)
    cnn_model = Model(cnn_inputs, x_cnn)

    combined = Concatenate()([ff_model.output, cnn_model.output])
    x_conc = combined
    for i in range(n_dense_layers):
        units = trial.suggest_int(f"combined_dense_units_{i}", 8, 256)
        x_conc = Dense(units, activation=activation_fn, kernel_initializer=initializer)(x_conc)
        x_conc = BatchNormalization()(x_conc)
        x_conc = Dropout(dropout)(x_conc)

    output = Dense(1, activation='linear', kernel_initializer=initializer)(x_conc)

    if optimizer_name == "adam":
        opt = tf.keras.optimizers.Adam(learning_rate=lr)
    else:
        wd = trial.suggest_float("weight_decay", 1e-6, 1e-3, log=True)
        opt = tf.keras.optimizers.AdamW(learning_rate=lr, weight_decay=wd)

    model = Model(inputs=[ff_model.input, cnn_model.input], outputs=output)
    model.compile(
        optimizer=opt,
        loss='mean_squared_error',
        metrics=[
            tf.keras.metrics.MeanAbsoluteError(name='mae'),
            tf.keras.metrics.MeanSquaredError(name='mse'),
            tf.keras.metrics.RootMeanSquaredError(name='rmse'),
            tf.keras.metrics.MeanAbsolutePercentageError(name='mape'),
            tf.keras.metrics.R2Score(name='r2_score'),
        ]
    )
    return model

def one_cycle_schedule(epoch, lr, total_epochs, max_lr, min_lr=1e-5):
    if epoch < total_epochs * 0.25:
        return min_lr + (max_lr - min_lr) * (epoch / (total_epochs * 0.25))
    else:
        progress = (epoch - total_epochs * 0.25) / (total_epochs * 0.75)
        return max_lr * 0.5 * (1 + np.cos(np.pi * progress))


def objective(trial):
    os.makedirs(base_checkpoint_dir, exist_ok=True)

    checkpoint_path = os.path.join(base_checkpoint_dir, f"trial_{trial.number}_best_model.keras")

    shape = len(X_train_num.columns)
    pixels = X_train_img.shape[1]

    model = create_multimodal_regressor(trial, shape, pixels)
    batch_size = trial.suggest_categorical("batch_size", [32])
    epochs = 200

    max_lr = trial.params.get("lr", 1e-2)
    lr_scheduler = tf.keras.callbacks.LearningRateScheduler(
        lambda epoch, lr: one_cycle_schedule(epoch, lr, total_epochs=epochs, max_lr=max_lr),
        verbose=0
    )

    checkpoint_cb = tf.keras.callbacks.ModelCheckpoint(
        checkpoint_path,
        monitor='val_loss',
        save_best_only=True,
        save_weights_only=False,
        mode='min',
        verbose=0
    )

    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor='val_loss', patience=6, restore_best_weights=True
    )

    history = model.fit(
        [X_train_num, X_train_img], y_train,
        validation_data=([X_val_num, X_val_img], y_val),
        epochs=epochs,
        batch_size=batch_size,
        verbose=1,
        callbacks=[early_stopping, checkpoint_cb, lr_scheduler]
    )

    val_loss = min(history.history['val_loss'])
    trial.set_user_attr("best_model_path", checkpoint_path)
    return val_loss

---
# <font color="#004D7F" size=5> 4.2. Compile and fit</font>

In [ ]:
force_retrain = True

if not os.path.exists(model_path) or force_retrain:

    print(f"Creating or loading study: {study_name} from {study_db_path}")
    study = optuna.create_study(
        study_name=study_name,
        storage=storage_url,
        direction="minimize",
        load_if_exists=True
    )

    n_total_trials = 75
    print(f"Current trials: {len(study.trials)}. Optimizing up to {n_total_trials} total trials.")
    study.optimize(objective, n_trials=(n_total_trials - len(study.trials)))

    print("\nOptimization complete. Best trial:")
    best_trial = study.best_trial
    print(f"  Value (val_loss): {best_trial.value}")
    for key, value in best_trial.params.items():
        print(f"    {key}: {value}")

    best_model_path = best_trial.user_attrs["best_model_path"]
    print(f"Loading best model from: {best_model_path}")
    best_model = load_model(best_model_path)

    best_model.save(model_path)
    model = best_model
    print(f"Best model saved to: {model_path}")

else:
    print(f"Model already exists at {model_path}. Loading it.")
    model = load_model(model_path)

print("Process finished.")

In [ ]:
plot_model(
    model,
    show_shapes=True,
    show_layer_names=True,
    expand_nested=True,
    dpi=96
)

---
# <font color="#004D7F" size=6> 5. Results</font>

---
# <font color="#004D7F" size=5> 5.1. Validation/Test evaluation</font>

In [ ]:
score_test = model.evaluate([X_val_num, X_val_img], y_val)

In [ ]:
score_test = model.evaluate([X_test_num, X_test_img], y_test)

In [ ]:
score_test

---
# <font color="#004D7F" size=6> 6. PermGrad Framework application</font>

---
## <font color="#004D7F" size=6> 6.1. Permutation Feature Importance - MLP</font>

In [ ]:
def plot_feature_importance_bar(feature_importance, title, output_path, top_n=10):
    sorted_features = sorted(feature_importance.items(), key=lambda item: item[1], reverse=True)

    top_features_list = sorted_features[:top_n]

    features, importances = zip(*top_features_list)
    print(f"Top {top_n} Features: {dict(top_features_list)}")

    y_positions = np.arange(len(features))

    plt.figure(figsize=(8, 6))

    cmap = plt.get_cmap("Accent")
    colors = [cmap(0)] * len(sorted_features)

    plt.barh(
        y_positions,
        importances,
        height=0.8,
        color=colors,
        edgecolor='white'
    )

    plt.axvline(x=0, color='grey', linestyle='--', linewidth=0.8)
    plt.ylabel("Features", fontsize=18)
    plt.xlabel("Feature Importance", fontsize=18)
    if title != "":
        plt.title(f"{title}", fontsize=18)

    plt.xticks(fontsize=14)
    plt.yticks(y_positions, [k for k,_ in sorted_features], size=14)

    plt.gca().invert_yaxis()

    plt.tight_layout()
    plt.savefig(output_path)
    plt.show()
    plt.close()

In [ ]:
if dataset == "Puma":
  dataset_path = f"../datasets/{dataset}"
  mlp_importances_named_path = f'{dataset_path}/mlp_metrics.npy'

n_repeats = 5

if not os.path.exists(mlp_importances_named_path):
    y_pred = model.predict([X_test_num, X_test_img], verbose=0).ravel()
    baseline_loss = tf.keras.losses.MSE(y_test, y_pred).numpy().mean()

    mlp_importances = {}

    for feature in tqdm(X_test_num.columns, desc="Permuting features"):
        deltas = []

        for _ in range(n_repeats):
            X_perm = X_test_num.copy()
            X_perm[feature] = np.random.permutation(X_perm[feature].values)

            y_pred_perm = model.predict([X_perm, X_test_img], verbose=0).ravel()

            loss_perm = tf.keras.losses.MSE(y_test, y_pred_perm).numpy().mean()
            deltas.append(loss_perm - baseline_loss)

        mlp_importances[feature] = np.mean(deltas)

    np.save(mlp_importances_named_path, mlp_importances, allow_pickle=True)

else:
    mlp_importances = np.load(mlp_importances_named_path, allow_pickle=True).item()

print(f"\n=== MLP‐branch ΔMSE per feature (averaged over {n_repeats} repeats) ===")
for f, imp in mlp_importances.items():
    print(f"{f}: {imp:.4f}")

In [ ]:
mlp_importances = {f: (v.numpy() if hasattr(v, "numpy") else v) for f, v in mlp_importances.items()}

min_val, max_val = min(mlp_importances.values()), max(mlp_importances.values())
mlp_importances_normalized = {f: (v - min_val) / (max_val - min_val) for f, v in mlp_importances.items()}

plot_feature_importance_bar(mlp_importances_normalized, title="", output_path = f'{dataset_path}/mlp_metrics.pdf')

---
## <font color="#004D7F" size=6> 6.2. Grad-CAM Heatmap Feature Importance - CNN</font>

In [ ]:
def cuadrado(coord):
    m = np.mean(coord, axis=0).reshape((1, 2))
    coord_nuevo = coord - m
    dista = (coord_nuevo[:, 0]**2 + coord_nuevo[:, 1]**2)**0.5
    maxi = math.ceil(max(dista))
    vertices = np.array([[-maxi, maxi], [-maxi, -maxi], [maxi, -maxi], [maxi, maxi]])
    coord_nuevo = coord_nuevo - vertices[0]
    vertices = vertices - vertices[0]
    return coord_nuevo, vertices

def m_imagen(coord, vertices, pixeles=20):
    size = (pixeles, pixeles)
    matriz = np.zeros(size)
    coord_m = (coord / vertices[2, 0]) * (pixeles - 1)
    coord_m = np.round(abs(coord_m))
    for i, j in zip(coord_m[:, 1], coord_m[:, 0]):
        matriz[int(i), int(j)] = 1
    if np.count_nonzero(matriz != 0) != coord.shape[0]:
        return coord_m, matriz, True
    else:
        return coord_m, matriz, False


class DataImg:
    def __init__(self, algoritmo='PCA', pixeles=20, seed=20, veces=4, amp=np.pi, distancia=0.1, pasos=4, opcion='maximo'):
        self.algoritmo = algoritmo
        self.p = pixeles
        self.seed = seed
        self.veces = veces

        self.amp = amp
        self.distancia = distancia
        self.pasos = pasos
        self.opcion = opcion

        self.error_pos = False

    def ObtenerCoord(self, X):
        self.min_max_scaler = MinMaxScaler()
        X = self.min_max_scaler.fit_transform(X)
        labels = np.arange(X.shape[1])
        X_trans = X.T


        if(self.algoritmo=='PCA'):
            X_embedded = PCA(n_components=2,random_state=self.seed).fit(X_trans).transform(X_trans)
        elif(self.algoritmo=='t-SNE'):
            for _ in range(self.veces):
                X_trans = np.append(X_trans,X_trans,axis=0)
                labels = np.append(labels,labels,axis=0)
            X_embedded = TSNE(n_components=2,random_state=self.seed,perplexity=50).fit_transform(X_trans)
        else:
            X_embedded = np.random.rand(X.shape[1],2)

        datos_coordenadas = {'x':X_embedded[:,0], 'y':X_embedded[:,1], 'Sector':labels}
        dc = pd.DataFrame(data=datos_coordenadas)
        self.coord_obtenidas = dc.groupby('Sector').mean().values

        del X_trans
        gc.collect()

    def Delimitacion(self):
        self.coordenadas_iniciales, self.vertices = cuadrado(self.coord_obtenidas)

    def ObtenerMatrizPosiciones(self, columns_names):
        self.pos_pixel_caract, self.m, self.error_pos = m_imagen(self.coordenadas_iniciales,self.vertices,pixeles=self.p)
        self.columns_coords = dict()

        for coord, column_name in zip(self.pos_pixel_caract, columns_names):
          self.columns_coords[column_name] = coord

        print(self.columns_coords)

    def Entrenamiento(self, X, columns_names):
        self.columns_names = columns_names
        self.ObtenerCoord(X)
        self.Delimitacion()
        self.ObtenerMatrizPosiciones(columns_names)

    def CrearImagenSinteticaConColores(self, pixels=20, column_names=None):
        matriz = np.ones((pixels, pixels, 3))

        colores = mpl.colormaps['tab10'](np.linspace(0, 1, len(column_names)))

        if hasattr(self, 'pos_pixel_caract') and self.pos_pixel_caract is not None and column_names is not None:
            for i, pos in enumerate(self.pos_pixel_caract):
                if i < len(column_names):
                    x, y = int(pos[0]), int(pos[1])
                    if x < pixels and y < pixels:
                        matriz[x, y] = colores[i][:3]

        patches = [mpatches.Patch(color=colores[i][:3], label=column_names[i]) for i in range(len(column_names))]

        plt.imshow(matriz)
        plt.legend(handles=patches, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
        plt.show()

modeloIMG = DataImg(algoritmo="PCA", pixeles=pixels, amp=np.pi, distancia=2, pasos=4, opcion='mean', seed=42, veces=4)

df_x_wo_img = df_x.copy().drop(["images","values"], axis=1)

df_x_sec = df_x_wo_img
modeloIMG.Entrenamiento(df_x_sec, df_x_sec.columns.values)
modeloIMG.CrearImagenSinteticaConColores(pixels=pixels, column_names=df_x_sec.columns.values)
modeloIMG.columns_coords

In [ ]:
def make_gradcam_heatmap(img_array, model, last_conv_layer_name):
    grad_model = keras.models.Model(
        model.inputs,
        [model.get_layer(last_conv_layer_name).output, model.output]
    )

    with tf.GradientTape() as tape:
        conv_output, prediction = grad_model(img_array)
        loss = prediction[:, 0]

    grads = tape.gradient(loss, conv_output)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    conv_output = conv_output[0]
    heatmap = conv_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    heatmap = tf.maximum(heatmap, 0)
    max_val = tf.reduce_max(heatmap)
    if max_val > 0:
        heatmap /= max_val

    return heatmap.numpy()


def get_gradcam_feature_importance(
    X_num,
    X_img,
    model,
    last_conv_layer_name,
    column_coords
):
    feature_sums = {col: 0.0 for col in X_num.columns}
    n_samples = len(X_num)
    pixels = X_img.shape[1]

    for i in tqdm(range(n_samples), desc="Grad‑CAM regression"):
        num_in = np.expand_dims(X_num.iloc[i].values, axis=0)
        img_in = np.expand_dims(X_img[i], axis=0)
        heatmap = make_gradcam_heatmap([num_in, img_in], model, last_conv_layer_name)

        h, w = heatmap.shape

        for col, (x, y) in column_coords.items():
            hx = int(x / pixels * w)
            hy = int(y / pixels * h)
            hx = np.clip(hx, 0, w - 1)
            hy = np.clip(hy, 0, h - 1)

            feature_sums[col] += heatmap[hy, hx]

    feature_importance = {col: feature_sums[col] / n_samples for col in feature_sums}
    return feature_importance

cnn_metrics_path = f'{dataset_path}/cnn_metrics.npy'

if not os.path.exists(cnn_metrics_path):
  heatmap_metrics = get_gradcam_feature_importance(
      X_test_num,
      X_test_img,
      model,
      'conv2d_1',
      modeloIMG.columns_coords
  )
  np.save(cnn_metrics_path, heatmap_metrics, allow_pickle=True)
else:
  heatmap_metrics = np.load(cnn_metrics_path, allow_pickle=True).item()

In [ ]:
heatmap_metrics = {f: (v.numpy() if hasattr(v, "numpy") else v) for f, v in heatmap_metrics.items()}

min_val, max_val = min(heatmap_metrics.values()), max(heatmap_metrics.values())
heatmap_metrics_normalized = {f: (v - min_val) / (max_val - min_val) for f, v in heatmap_metrics.items()}

plot_feature_importance_bar(heatmap_metrics_normalized, title="Grad-CAM Heatmap Feature Importance", output_path=f'{dataset_path}/cnn_metrics.pdf')

In [ ]:
def display_image_with_labels(ax, image, column_coords,
                              max_labels_per_pixel=1,
                              stack_spacing=0.5,
                              max_label_length=11, fontsize=14):
    pixels = image.shape[0]
    ax.imshow(image)

    pixel_map = {}
    short_to_long = {}

    i = 1
    for label, coords in column_coords.items():
        row, col = int(coords[0]), int(coords[1])
        if 0 <= row < pixels and 0 <= col < pixels:
            vname = f"V{i}"
            pixel_map.setdefault((row, col), []).append((vname, label))
            short_to_long[vname] = label
            i += 1

    def _clamp(v, lo, hi):
        return max(lo, min(hi, v))

    cluster_id = 0
    cluster_map = {}

    for (row, col), labels in pixel_map.items():
        k = len(labels)

        if k <= max_labels_per_pixel:
            start_y = row - (k - 1) * stack_spacing / 2.0
            for i, (short_name, long_name) in enumerate(labels):
                label_text = short_name
                if len(label_text) > max_label_length:
                    label_text = label_text[:max_label_length-1] + "…"

                plot_y = start_y + i * stack_spacing
                clamped_x = _clamp(col, -0.5, pixels - 0.5)
                clamped_y = _clamp(plot_y, -0.5, pixels - 0.5)

                ax.text(clamped_x, clamped_y - 1, label_text,
                        ha='center', va='center', fontsize=fontsize, zorder=10,
                        bbox=dict(facecolor='white', alpha=0.45, edgecolor='none', pad=0.6),
                        clip_on=True)

        else:
            cluster_name = f"C{cluster_id}"
            cluster_contents = [long for _, long in labels]
            cluster_map[cluster_name] = cluster_contents

            ax.text(col, row - 1, cluster_name,
                    ha='center', va='center', fontsize=fontsize, fontweight='bold', zorder=10,
                    bbox=dict(facecolor='white', alpha=0.45, edgecolor='none', pad=0.8),
                    clip_on=True)

            cluster_id += 1

    ax.set_xticks([])
    ax.set_yticks([])

    print("\n--- Label Translation Dictionary ---")
    for v, full in short_to_long.items():
        print(f"{v} -> {full}")
    for c, full_list in cluster_map.items():
        print(f"{c} -> {', '.join(full_list)}")
    print("------------------------------------")

    return short_to_long, cluster_map

fig, ax = plt.subplots(1, 1, figsize=(4, 4))
sample_image = X_test_img[10]
display_image_with_labels(ax, sample_image, modeloIMG.columns_coords, max_labels_per_pixel=1)
plt.tight_layout()
plt.savefig(f'{dataset_path}/tinto_sample.pdf')
plt.show()

In [ ]:
def save_and_display_gradcam(img, heatmap, cam_path="cam.jpg", alpha=0.4):
    img = keras.preprocessing.image.img_to_array(img)

    heatmap = tf.maximum(heatmap, 0)
    max_val = tf.reduce_max(heatmap)
    if max_val > 0:
        heatmap = heatmap / max_val
    heatmap = np.uint8(255 * heatmap.numpy())

    jet = plt.get_cmap("jet")
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

    superimposed_img = jet_heatmap * alpha + img
    superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)

    superimposed_img.save(cam_path)
    return superimposed_img


def generate_heatmap_and_image(X_val_num, X_val_img, modelX, layer_name, model_type="HYBRID"):
    if model_type == "HYBRID":
        input_data = [np.expand_dims(X_val_num, axis=0), np.expand_dims(X_val_img, axis=0)]
    elif model_type == "CNN":
        input_data = np.expand_dims(X_val_img, axis=0) / 255.0
    else:
        raise ValueError("model_type must be 'HYBRID' or 'CNN'.")

    heatmap = make_gradcam_heatmap(input_data, modelX, layer_name)

    return save_and_display_gradcam(X_val_img, heatmap)


def display_image(ax, row, col, image, title):
    axis = ax[row][col] if ax.ndim > 1 else ax[col]
    axis.imshow(image)
    axis.set_title(title, fontsize=14)
    axis.set_xticks([])
    axis.set_yticks([])

In [ ]:
samples = 1
fig, ax = plt.subplots(1, 2, figsize=(9, 4), squeeze=False)
i = 10

X_test_data = []
label = "Original Image"
X_test_data.append((X_test_num.iloc[i], X_test_img[i], label))
num, img, title = X_test_data[0]
display_image(ax, 0, 0, img, title)

layer = "conv2d_1"
superimposed_img = generate_heatmap_and_image(num, img, model, layer, "HYBRID")
display_title = "Grad-CAM"
display_image(ax, 0, 1, superimposed_img, display_title)

jet_cmap = plt.colormaps['jet']

jet_colors = jet_cmap(range(256))
jet_colors[:, -1] = 0.4
transparent_jet = colors.ListedColormap(jet_colors)

norm = colors.Normalize(vmin=0, vmax=1)
sm = cm.ScalarMappable(cmap=transparent_jet, norm=norm)
sm.set_array([])

cbar = fig.colorbar(sm, ax=ax[0, 1], orientation='vertical', fraction=0.046, pad=0.04)

plt.tight_layout()
plt.savefig(f'{dataset_path}/feature_importance_sample.pdf')
plt.show()

---
## <font color="#004D7F" size=6> 6.3. PermGrad Feature Importance</font>

In [ ]:
joint_importances_named_path = f'{dataset_path}/joint_metrics.npy'

if not os.path.exists(joint_importances_named_path):
  joint_metrics = {}

  print("--- Calculating Baseline Performance ---")
  y_probas_baseline = model.predict([X_test_num, X_test_img], verbose=0)
  baseline_loss = np.mean(
      tf.keras.losses.MSE(y_test, y_probas_baseline)
  )
  print(f"Baseline test loss: {baseline_loss:.4f}\n")

  numeric_input, image_input = model.inputs

  mlp_output = model.get_layer('dropout_17').output
  cnn_output = model.get_layer('flatten_2').output

  def forward_from(layer_name, new_input):
      x = new_input
      found = False
      for layer in model.layers:
          if layer.name == layer_name:
              found = True
              continue
          if found:
              x = layer(x)
      return x

  zeros_for_cnn = Lambda(lambda x: tf.zeros_like(x), name='zeros_for_cnn')(cnn_output)
  combined_cnn = model.get_layer('concatenate_2')([mlp_output, zeros_for_cnn])

  final_output_cnn = forward_from('concatenate_2', combined_cnn)
  ablation_model_cnn = Model(
      inputs=[numeric_input, image_input], outputs=final_output_cnn
  )

  y_probas_cnn = ablation_model_cnn.predict([X_test_num, X_test_img], verbose=0)
  loss_cnn = np.mean(tf.keras.losses.MSE(y_test, y_probas_cnn).numpy())
  joint_metrics["CNN"] = float(loss_cnn - baseline_loss)

  print("CNN branch ablation:")
  print(f"  Ablated loss: {loss_cnn:.4f}")
  print(f"  ΔLoss (Change in Loss): {joint_metrics['CNN']:.4f}\n")

  zeros_for_mlp = Lambda(lambda x: tf.zeros_like(x), name='zeros_for_mlp')(mlp_output)
  combined_mlp = model.get_layer('concatenate_2')([zeros_for_mlp, cnn_output])

  final_output_mlp = forward_from('concatenate_2', combined_mlp)
  ablation_model_mlp = Model(
      inputs=[numeric_input, image_input], outputs=final_output_mlp
  )

  y_probas_mlp = ablation_model_mlp.predict([X_test_num, X_test_img], verbose=0)
  loss_mlp = np.mean(tf.keras.losses.MSE(y_test, y_probas_mlp).numpy())
  joint_metrics["MLP"] = float(loss_mlp - baseline_loss)

  print("MLP branch ablation:")
  print(f"  Ablated loss: {loss_mlp:.4f}")
  print(f"  ΔLoss (Change in Loss): {joint_metrics['MLP']:.4f}\n")

  print("ΔLoss Results Dictionary:")
  print(joint_metrics)

  all_values = list(joint_metrics.values())
  den = sum(math.exp(v) for v in all_values)
  joint_metrics = {b: math.exp(v) / den for b, v in joint_metrics.items()}

  np.save(joint_importances_named_path, joint_metrics, allow_pickle=True)
else:
  joint_metrics = np.load(joint_importances_named_path, allow_pickle=True).item()

In [ ]:
print("Branch Importance:")
print(joint_metrics)

In [ ]:
perm_grad_mlp_importances_normalized = {
    feature: joint_metrics["MLP"] * val
                  for feature, val in mlp_importances_normalized.items()
}

perm_grad_heatmap_metrics_normalized = {
    feature: joint_metrics["CNN"] * val
                  for feature, val in heatmap_metrics_normalized.items()
}

global_importance_by_branch = {}

for feature in perm_grad_mlp_importances_normalized.keys():
    global_importance_by_branch[feature] = perm_grad_mlp_importances_normalized[feature] + perm_grad_heatmap_metrics_normalized[feature]

In [ ]:
plot_feature_importance_bar(global_importance_by_branch, title="", output_path=f'{dataset_path}/permgrad.pdf')

In [ ]:
global_importance_by_branch

result = {}
for key, value in global_importance_by_branch.items():
    result[key] = result.get(key, 0) + value

result